### Environment
`conda create  --name eleven-env python=3.8.16`  
`conda activate eleven-env`  
`/Users/Antoine/opt/anaconda3/envs/eleven-env/bin/pip install -r requirements.txt`

### Import libraries

In [1]:
import pandas as pd
from pathlib import Path
import os
import datetime
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

In [2]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.absolute()
data_path = os.path.join(root_path, 'data')
os.chdir(root_path)

In [3]:
attendance = pd.read_csv(os.path.join(data_path, 'attendance.csv'))
entity_schedule = pd.read_csv(os.path.join(data_path, 'entity_schedule.csv'))
link_attraction_park = pd.read_csv(os.path.join(data_path, 'link_attraction_park.csv'))
#parade_night_show = pd.read_csv(os.path.join(data_path, 'parade_night_show.xlsx'))
waiting_times = pd.read_csv(os.path.join(data_path, 'waiting_times.csv'))
weather_data = pd.read_csv(os.path.join(data_path, 'weather_data.csv'))

In [8]:
att = attendance.copy()
att['year'] = pd.to_datetime(attendance.USAGE_DATE).apply(lambda x: x.year)
att[att.year==2019].head()


,USAGE_DATE,FACILITY_NAME,attendance,year
428,2019-01-01,PortAventura World,57824,2019
429,2019-01-01,Tivoli Gardens,30185,2019
430,2019-01-02,PortAventura World,62595,2019
431,2019-01-02,Tivoli Gardens,30794,2019
432,2019-01-03,PortAventura World,66944,2019


In [41]:
waiting_times.head()

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
0,2018-01-01,2018-01-01 21:00:00.000,21,2018-01-01 21:15:00.000,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0
1,2018-01-01,2018-01-01 19:30:00.000,19,2018-01-01 19:45:00.000,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0
2,2018-01-01,2018-01-01 22:30:00.000,22,2018-01-01 22:45:00.000,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0
3,2018-01-01,2018-01-01 12:45:00.000,12,2018-01-01 13:00:00.000,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0
4,2018-01-01,2018-01-01 17:00:00.000,17,2018-01-01 17:15:00.000,Skyway,5,15.0,92.0,211.500,198.25,15,15,0,16.0


In [47]:
to_plot = waiting_times[["WAIT_TIME_MAX", "ENTITY_DESCRIPTION_SHORT"]].groupby("ENTITY_DESCRIPTION_SHORT").mean().sort_values('WAIT_TIME_MAX', ascending=False).reset_index()

In [48]:
to_plot.head()

,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX
0,Monorail,35.387042
1,Spiral Slide,32.245107
2,Giant Wheel,31.422579
3,Swing Ride,29.313063
4,Tilt-A-Whirl,27.993695


In [49]:
import altair as alt

alt.Chart(to_plot).mark_bar().encode(
x=alt.X('ENTITY_DESCRIPTION_SHORT', sort=None),
y='WAIT_TIME_MAX',
)

alt.Chart(...)

In [5]:
attendance.head(2)

,USAGE_DATE,FACILITY_NAME,attendance
0,2018-06-01,PortAventura World,46804
1,2018-06-01,Tivoli Gardens,20420


In [38]:
# Average anual attendance
to_plot_month = attendance.copy()
to_plot_month["month"] = pd.to_datetime(attendance.USAGE_DATE).apply(lambda x: x.month)
to_plot_month = to_plot_month[["FACILITY_NAME", "month", "attendance"]].groupby(["FACILITY_NAME", "month"]).mean().reset_index()
mapping = {1: 'january',\
    2: 'february',
    3: 'march',
    4: 'april',
    5: 'may',
    6: 'june',
    7: 'july',
    8: 'august',
    9: 'september',
    10: 'october',
    11: 'november',
    12: 'december'}
to_plot_month = to_plot_month.replace({'month': mapping})

to_plot_day = attendance.copy()
to_plot_day["day"] = pd.to_datetime(attendance.USAGE_DATE).apply(lambda x: x.weekday())
to_plot_day = to_plot_day[["FACILITY_NAME", "day", "attendance"]].groupby(["FACILITY_NAME", "day"]).mean().reset_index()
mapping = {0: 'monday',\
    1: 'tuesday',
    2: 'wednesday',
    3: 'thursday',
    4: 'friday',
    5: 'saturday',
    6: 'sunday'}
to_plot_day = to_plot_day.replace({'day': mapping})

In [39]:
to_plot_month_1 = to_plot_month[to_plot_month.FACILITY_NAME=="Tivoli Gardens"]
to_plot_month_2 = to_plot_month[to_plot_month.FACILITY_NAME=="PortAventura World"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=to_plot_month_1.month, y=to_plot_month_1.attendance, mode="lines", name="Tivoli Gardens", line=dict(color='#002244')))
fig.add_trace(go.Scatter(x=to_plot_month_2.month, y=to_plot_month_2.attendance, mode="lines", name="PortAventura World", line=dict(color='#ff0066')))
fig.update_layout(yaxis_title='Attendance', width=800, height=400)
fig.show()

In [40]:
to_plot_day_1 = to_plot_day[to_plot_day.FACILITY_NAME=="Tivoli Gardens"]
to_plot_day_2 = to_plot_day[to_plot_day.FACILITY_NAME=="PortAventura World"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=to_plot_day_1.day, y=to_plot_day_1.attendance, mode="lines", name="Tivoli Gardens", line=dict(color='#002244')))
fig.add_trace(go.Scatter(x=to_plot_day_2.day, y=to_plot_day_2.attendance, mode="lines", name="PortAventura World", line=dict(color='#ff0066')))
fig.update_layout(yaxis_title='Attendance', width=800, height=400)
fig.show()

In [6]:
entity_schedule.head(2)

,REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE
0,NaN,Tivoli Gardens,PARK,2018-11-19 10:00:00.000,2018-11-19 18:00:00.000,2018-11-20 08:24:32.000,2018-11-19
1,NaN,Dizzy Dropper,ATTR,2022-04-07 08:30:00.000,2022-04-07 22:04:00.000,2022-04-08 08:00:30.000,2022-04-07


In [11]:
link_attraction_park.head(2)

,ATTRACTION;PARK
0,Aeroplane Ride;Tivoli Gardens
1,Bumper Cars;PortAventura World


In [18]:
l2 = link_attraction_park.copy()

l2["Attraction"] = l2["ATTRACTION;PARK"].apply(lambda x: x.split(";")[0])
l2["Park"] = l2["ATTRACTION;PARK"].apply(lambda x: x.split(";")[1])
l2 = l2.drop("ATTRACTION;PARK", axis=1)
l2.groupby("Park").agg(list).reset_index()
l2.head(2)

,Attraction,Park
0,Aeroplane Ride,Tivoli Gardens
1,Bumper Cars,PortAventura World


,Park,Attraction
0,PortAventura World,"[Bumper Cars, Bungee Jump, Circus Train, Crazy..."
1,Tivoli Gardens,"[Aeroplane Ride, Crazy Bus, Gondola, Log Flume..."


In [24]:
to_plot = pd.DataFrame()
for idx, park in enumerate(l2.Park):
    to_plot[f"{park}"] = l2.loc[idx, "Attraction"]
to_plot.head()

,Tivoli Gardens,PortAventura World


In [9]:
waiting_times.head(2)

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
0,2018-01-01,2018-01-01 21:00:00.000,21,2018-01-01 21:15:00.000,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0
1,2018-01-01,2018-01-01 19:30:00.000,19,2018-01-01 19:45:00.000,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0


In [8]:
weather_data.head(2)

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,915148800,1999-01-01 00:00:00 +0000 UTC,3600,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,...,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n
1,915152400,1999-01-01 01:00:00 +0000 UTC,3600,Custom location,48.873492,2.295104,8.08,NaN,3.54,5.18,...,NaN,NaN,NaN,NaN,NaN,6,800,Clear,sky is clear,01n


### Attendance evolution

In [4]:
print(attendance.shape)
attendance["USAGE_DATE"] = pd.to_datetime(attendance.USAGE_DATE)
attendance.head(2)

(2367, 3)


,USAGE_DATE,FACILITY_NAME,attendance
0,2018-06-01,PortAventura World,46804
1,2018-06-01,Tivoli Gardens,20420


In [5]:
start_date = pd.to_datetime('2018/06/01')
end_date = pd.to_datetime('2019/06/01')
start_date, end_date

(Timestamp('2018-06-01 00:00:00'), Timestamp('2019-06-01 00:00:00'))

In [6]:
to_plot = attendance.copy()
to_plot = to_plot[(to_plot.USAGE_DATE>start_date)&(to_plot.USAGE_DATE<end_date)][to_plot.FACILITY_NAME=="Tivoli Gardens"]
to_plot.head(2)

/var/folders/q6/rqx7by952vl927kfl2fslt5h0000gn/T/ipykernel_14410/3856638061.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  to_plot = to_plot[(to_plot.USAGE_DATE>start_date)&(to_plot.USAGE_DATE<end_date)][to_plot.FACILITY_NAME=="Tivoli Gardens"]


,USAGE_DATE,FACILITY_NAME,attendance
3,2018-06-02,Tivoli Gardens,29110
5,2018-06-03,Tivoli Gardens,23727


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=to_plot["USAGE_DATE"], y=to_plot["attendance"], mode="lines", name='Attendance', line=dict(color='#002244')))
fig.update_layout(yaxis_title='CPU usage', width=800, height=400)
fig.show()

In [25]:
def attendance_figures(df, attraction_list, start_date, end_date, date_label, attendance_label, attraction_label):
    '''
    Return the attendance mean, max and min. 

    Inputs:
        df : pd.DataFrame() = table of the attendance per date and per attraction. 
        attraction_list : list(str) = list of the names of the attractions.
        start_date : str = starting date of the period with format yyyy/mm/dd.
        end_date : str = ending date of the period with format yyyy/mm/dd.
        date_label : str = label of the date column.
        attendance_label : str = mabel of the attendance column.
        attraction_label : str = label of the attraction column.

    Outputs:
        mean : int = average daily attendnace of the attraction over the period.
        min : min daily attendance at the attraction during the period.
        max : max daily attendance at the attraction during the perdiod. 
    '''
    to_plot = df.copy()

    # Change date types
    to_plot[f"{date_label}"] = pd.to_datetime(to_plot[f"{date_label}"])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter the table
    to_plot = to_plot[(to_plot[f"{date_label}"]>start_date)&(to_plot[f"{date_label}"]<end_date)][to_plot[f"{attraction_label}"].isin(attraction_list)]

    # Calculate figures
    val_min = to_plot[f"{attendance_label}"].min()
    val_avg = to_plot[f"{attendance_label}"].mean()
    val_max = to_plot[f"{attendance_label}"].max()

    # Generate plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=to_plot["USAGE_DATE"], y=to_plot["attendance"], mode="lines", name='Attendance', line=dict(color='#002244')))
    fig.update_layout(yaxis_title='CPU usage', width=800, height=400)
    
    return fig, [val_min, val_avg, val_max]

In [26]:
df = attendance
attraction = ["Tivoli Gardens"]
start_date = '2018/06/01'
end_date = '2019/06/01'
date_label = "USAGE_DATE"
attendance_label = "attendance"
attraction_label = "FACILITY_NAME"

In [27]:
fig, L = attendance_figures(df, attraction, start_date, end_date, date_label, attendance_label, attraction_label)
print('min', 'mean', 'max')
print(L)

min mean max
[5320, 22382.684065934067, 44052]


/var/folders/q6/rqx7by952vl927kfl2fslt5h0000gn/T/ipykernel_14410/2090608957.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [28]:
fig.show()

In [4]:
df = pd.read_csv(os.path.join(data_path, 'all_data_clean.csv'))
df.head()

/var/folders/q6/rqx7by952vl927kfl2fslt5h0000gn/T/ipykernel_17005/1204390691.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_path, 'all_data_clean.csv'))


,work_date,deb_time_line,fin_time_line,entity_description_short,wait_time_max,nb_units,guest_carried,capacity,adjust_capacity,open_time,...,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,clouds_all,weather_id,weather_main,weather_icon,summer_time
0,2018-10-01 00:00:00,2018-10-01 16:30:00,2018-10-01 16:45:00,Roller Coaster,40,1.7333,25.0,75.000,65.00,15,...,334,0.0,0.0,0.0,0.0,91,804,Clouds,04d,True
1,2018-10-01 00:00:00,2018-10-01 16:45:00,2018-10-01 17:00:00,Roller Coaster,15,2.0000,25.0,75.000,75.00,15,...,334,0.0,0.0,0.0,0.0,91,804,Clouds,04d,True
2,2018-10-01 00:00:00,2018-10-01 16:15:00,2018-10-01 16:30:00,Roller Coaster,10,2.0000,29.0,75.000,75.00,15,...,334,0.0,0.0,0.0,0.0,91,804,Clouds,04d,True
3,2018-10-01 00:00:00,2018-10-01 16:00:00,2018-10-01 16:15:00,Roller Coaster,10,2.0000,30.0,75.000,75.00,15,...,334,0.0,0.0,0.0,0.0,91,804,Clouds,04d,True
4,2018-10-01 00:00:00,2018-10-01 16:30:00,2018-10-01 16:45:00,Bumper Cars,10,18.0000,144.0,254.749,254.75,15,...,334,0.0,0.0,0.0,0.0,91,804,Clouds,04d,True
